# [Python Scopes and Namespaces](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces)

A **namespace** is a mapping from names to objects. Most namespaces are currently implemented as *Python dictionaries*. Examples of namespaces are: the set of built-in names (containing functions such as `abs()`); the global names in a module; and the local names in a function invocation.

In a sense the set of attributes ([doc. definition](https://docs.python.org/3/tutorial/classes.html#python-scopes-and-namespaces): attributes = any name following a dot) of an object also form a namespace.

Namespace's attributes may be **read-only or writable**.

The important thing to know about namespaces is that there is absolutely **no relation between names in different namespaces**; for instance, two different modules may both define a function `maximize` without confusion — users of the modules must prefix it with the module name.

Namespaces are **created at different moments** and have different lifetimes. The namespace containing the built-in names is created when the Python interpreter starts up, and is never deleted. The global namespace for a module is created when the module is read in; normally, module namespaces also last until the interpreter quits. The local namespace for a function is created when the function is called, and deleted when the function returns.

A **scope is a textual region of a Python program where a namespace is directly accessible**. At any time during execution, there are 3 or 4 nested scopes whose namespaces are directly accessible:
1. the innermost scope, which is searched first, contains the local names
2. the scopes of any enclosing functions, which are searched starting with the nearest enclosing scope, contain non-local, but also non-global names
3. the next-to-last scope contains the current module's global names
4. the outermost scope (searched last) is the namespace containing built-in names.

If a name is declared `global`, then all references and assignments go directly to the next-to-last scope containing the module's global names. To rebind variables found outside of the innermost scope, the `nonlocal` statement can be used; if not declared `nonlocal`, those variables are read-only (an attempt to write to such a variable will simply create a new local variable in the innermost scope, leaving the identically named outer variable unchanged).

Note: outside functions, the local scope references the same namespace as the global scope: the module's namespace. It's also important to realize that scopes are determined textually: the global scope of a function defined in a module is that module's namespace.

A special quirk of Python is that – if no `global` or `nonlocal` statement is in effect – **assignments to names always go into the innermost scope**. Assignments do not copy data (like in C) — they **just bind names to objects**. The same is true for deletions: the statement `del x` removes the binding of `x` from the namespace referenced by the local scope. In fact, all operations that introduce new names use the local scope: in particular, `import` ([doc. definition](https://docs.python.org/3/glossary.html#term-importing): importing = The process by which Python code in one module is made available to Python code in another module.) statements and function definitions bind the module or function name in the local scope.

The `global` statement can be used to indicate that particular variables live in the global scope and should be rebound there; the `nonlocal` statement indicates that particular variables live in an enclosing scope and should be rebound there.

In [1]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


Note how the local assignment (which is default) didn’t change scope_test's binding of `spam`. The `nonlocal` assignment changed scope_test's binding of `spam`, and the `global` assignment changed the module-level binding.

You can also see that there was no previous binding for `spam` before the global assignment, so it we remove `do_global` an error will be raised (check code bellow).

In [2]:
def scope_test():
    def do_local():
        x = "local x"

    def do_nonlocal():
        nonlocal x
        x = "nonlocal x"

    x = "test x"
    do_local()
    print("After local assignment:", x)
    do_nonlocal()
    print("After nonlocal assignment:", x)

scope_test()
try:
    print("In global scope:", x)
except NameError as e:
    print("ERROR:", e)

After local assignment: test x
After nonlocal assignment: nonlocal x
ERROR: name 'x' is not defined


# [Modules and packages](https://docs.python.org/3/tutorial/modules.html#modules)

If you quit from the Python interpreter and enter it again, the definitions you have made (functions and variables) are lost. Therefore, if you want to write a somewhat longer program, you are better off using a text editor to prepare the input for the interpreter and running it with that file as input instead. This is known as creating a **script**. As your program gets longer, you may want to split it into several files for easier maintenance. Python solves this with [**modules**](https://docs.python.org/3/glossary.html#term-module) and [**packages**](https://docs.python.org/3/glossary.html#term-package), which let you organize code into logical pieces.

- **Module**: a file (`.py`) containing Python definitions and statements. The filename becomes the module name (note: the module's name (as a string) is available as the value of the global variable `__name__`). Definitions from a module can be imported into other modules.

In [3]:
import fibo
fibo

<module 'fibo' from '/Users/joaoloss/Documents/python-playground/fibo.py'>

The code above *does not* add the names of the functions defined in `fibo` directly to the current namespace; it only adds the module name fibo there. Using the module name you can access the functions (it's a nested namespace).

The imported module names, if placed at the top level of a module (outside any functions or classes), are added to the module’s global namespace.

A module can contain executable statements as well as function definitions. These statements are intended to initialize the module. They are executed only the first time the module name is encountered in an import statement. (They are also run if the file is executed as a script.)

`from fibo import fib` is a variant of the import statement that imports names from a module directly into the importing module’s namespace. This **does not** introduce the module name from which the imports are taken in the local namespace (so in the example, fibo is not defined).

There is even a variant to import all names that a module defines: `from fibo import *`. This imports all names except those beginning with an underscore (_). In most cases Python programmers do not use this facility since it introduces an unknown set of names into the interpreter, possibly hiding some things you have already defined. Note that in general the practice of importing `*` from a module or package is frowned upon, since it often causes poorly readable code. However, it is okay to use it to save typing in interactive sessions.

When you run a Python module is directly executed, such as: `python fibo.py <arguments>`, the code in the module will be executed, just as if you imported it, but with the `__name__` set to `"__main__"`.

When a module named `spam` is imported, the interpreter first searches for a built-in module with that name. If not found, it then searches for a file named `spam.py` in a list of directories given by the variable `sys.path`.

In [4]:
import sys
import builtins

In [5]:
dir(builtins)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BaseExceptionGroup',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'ExceptionGroup',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'PythonFinalizationError',
 'RecursionError',
 'ReferenceError',

In [6]:
sys.path

['/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python313.zip',
 '/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13',
 '/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload',
 '',
 '/Users/joaoloss/Documents/python-playground/.venv/lib/python3.13/site-packages']

To speed up loading modules, **Python caches the compiled version of each module** in the `__pycache__` directory under the name module.version.pyc, where the version encodes the format of the compiled file; it generally contains the Python version number. For example, in CPython release 3.3 the compiled version of spam.py would be cached as `__pycache__/spam.cpython-33.pyc`. This naming convention allows compiled modules from different releases and different versions of Python to coexist.

Python checks the modification date of the source against the compiled version to see if it’s out of date and needs to be recompiled. This is a completely automatic process. Also, the compiled modules are platform-independent

Built-in function `dir()`: Without arguments, return the list of names in the current local scope. With an argument, attempt to return a list of valid attributes for that object.

In [7]:
dir()

['In',
 'Out',
 '_',
 '_3',
 '_5',
 '_6',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'builtins',
 'exit',
 'fibo',
 'get_ipython',
 'open',
 'quit',
 'scope_test',
 'spam',
 'sys']

In [8]:
dir(fibo)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'fib']

- **Package**: are a way of structuring Python's module namespace by using "dotted module names". For example, the module name A.B designates a submodule named B in a package named A. Just like the use of modules saves the authors of different modules from having to worry about each other's global variable names, the use of dotted module names saves the authors of multi-module packages like NumPy or Pandas from having to worry about each other's module names.

Mental model:
- Package = folder
- Module = file

Before Python 3.3, a folder had to contain `__init__.py` to be considered a package. Today it’s optional, but still very useful as it can execute initialization code for the package or set the `__all__` variable.

*Note*: A **regular package** is a directory that contains an `__init__.py` file, which explicitly marks the directory as a package and allows initialization code to run, so that variables, classes, and functions can be imported directly from the package as if it were a module. A **namespace package**, by contrast, is a package without an `__init__.py` file and is used only for name resolution (as said in the definition above).

*Note*: `__all__` is a list of names that a module or package explicitly says are public and should be imported when someone uses `from module import *`.

In [9]:
import src # namespace package (no __init__.py)
src

<module 'src' (namespace) from ['/Users/joaoloss/Documents/python-playground/src']>

In [10]:
import src.pack # regular package (because of __init__.py)
src.pack

<module 'src.pack' from '/Users/joaoloss/Documents/python-playground/src/pack/__init__.py'>

In [11]:
import src.main
src.main

<module 'src.main' from '/Users/joaoloss/Documents/python-playground/src/main.py'>

Note: when using `from package import item`, `item` can be either a submodule (or subpackage) of the package, or some other name defined in the package, like a function, class or variable. However, when using syntax like `import item.subitem.subsubitem`, each item except for the last must be a package; the last item can be a module or a package but can't be a class or function or variable defined in the previous item.

In [ ]:
try:
    import src.main.demo_function # incorrect import as said above
except ModuleNotFoundError as e:
    print("ERROR:", e)

ERROR: No module named 'src.main.demo_function'; 'src.main' is not a package


In [13]:
from src.main import demo_function 
demo_function

<function src.main.demo_function()>